In [45]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"
os.environ["HF_HUB_DISABLE_PROGRESS_BAR"] = "0"

In [46]:
from unsloth import FastLanguageModel
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "novak247/massage_assistant_v1_lora", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

train_prompt_style = """Below is an instruction that describes a task for a massage robot, paired with an input that provides further context.
Write a response that generates an executable pipeline in Python using only the provided functions.
Before answering, analyze the task carefully and generate a clear sequence of commands.

### Instruction:
You are provided with high-level instructions for operating a massage robot. Create an executable pipeline in Python that structures task execution through a sub-task pipeline. This pipeline should be composed exclusively of the functions listed below in the Capabilities section, arranged in a logical and correct order so that it can be directly executed. Your response should only consist of the pipeline without additional information.

Capabilities:
    start() → Initializes the robot.
    stop() → Stops the robot.
    home() → Moves the robot to the home position.
    [x, y, z] = detect_body_part(part_name) → Detects the specified body part and returns the coordinates.
    move_to([x, y, z]) → Moves the robot to the specified coordinates.
    change_force(mode, value) → Adjusts the massage force based on the specified mode:
        - If mode is 'absolute', then value is any real number, setting the force directly.
        - If mode is 'relative', then value is between -1 and 1, modifying the current force F as:
          F_new = (1 + value) * F
    automatic_massage(part_name) → Automatically massages the specified body part.

### Question:
{}

### Response:
{}"""
question = "Would you be so kind and massage by lower back"
inputs = tokenizer([train_prompt_style.format(question, "")], return_tensors="pt").to("cuda")
print("Input prepared and moved to GPU.")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, max_new_tokens = 64, pad_token_id = tokenizer.eos_token_id)

==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.621 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Input prepared and moved to GPU.
[x, y, z] = detect_body_part('lower back')
move_to([x, y, z])<｜end▁of▁sentence｜>


In [54]:
question = "massage my shoulders, then lower back and then go home and stop"
inputs = tokenizer([train_prompt_style.format(question, "")], return_tensors="pt").to("cuda")
# print("Input prepared and moved to GPU.")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, max_new_tokens = 100, pad_token_id = tokenizer.eos_token_id)

[x, y, z] = detect_body_part('shoulders')
move_to([x, y, z])
automatic_massage('shoulders')
[x, y, z] = detect_body_part('lower back')
move_to([x, y, z])
automatic_massage('lower back')
home()
stop()<｜end▁of▁sentence｜>
